In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
import datetime 

In [3]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders
        self.order = None

        # parameters for indicatiors
        self.sma = bt.ind.SMA(period=20)
        self.boll = bt.ind.BollingerBands(period=20, devfactor=2.5)
        self.rsi = bt.ind.RSI(period=2, safediv=True)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.data.close < self.sma and \
            self.data.close < self.boll.lines.bot:
                
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy(exectype = bt.Order.Limit,
                        price = 0.97 * self.dataclose[0])

        else:

            # Already in the market ... we might sell
            if self.rsi > 50 or len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [16]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(1000.0)

       # Add a strategy
    cerebro.addstrategy(TestStrategy)

    datapath = '../data/AMZN.csv'

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date 2021-10-11
        fromdate=datetime.datetime(2021, 10, 11),
        # Do not pass values after this date
        todate=datetime.datetime(2022, 10, 11),
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, compression=1, factor=365, annualize=True)

    results = cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Sharpe Ratio: ', results[0].analyzers.sharperatio.get_analysis()['sharperatio'])
    

Starting Portfolio Value: 1000.00
2021-11-05, Close, 175.95
2021-11-08, Close, 174.45
2021-11-09, Close, 178.81
2021-11-10, Close, 174.10
2021-11-11, Close, 173.62
2021-11-12, Close, 176.26
2021-11-15, Close, 177.28
2021-11-16, Close, 177.04
2021-11-17, Close, 177.45
2021-11-18, Close, 184.80
2021-11-19, Close, 183.83
2021-11-22, Close, 178.63
2021-11-23, Close, 179.00
2021-11-24, Close, 179.02
2021-11-26, Close, 175.23
2021-11-29, Close, 178.08
2021-11-30, Close, 175.35
2021-12-01, Close, 172.19
2021-12-02, Close, 171.87
2021-12-03, Close, 169.49
2021-12-06, Close, 171.37
2021-12-07, Close, 176.16
2021-12-08, Close, 176.16
2021-12-09, Close, 174.17
2021-12-10, Close, 172.21
2021-12-13, Close, 169.57
2021-12-14, Close, 169.09
2021-12-15, Close, 173.32
2021-12-16, Close, 168.87
2021-12-17, Close, 170.02
2021-12-20, Close, 167.08
2021-12-21, Close, 170.42
2021-12-22, Close, 171.04
2021-12-23, Close, 171.07
2021-12-27, Close, 169.67
2021-12-28, Close, 170.66
2021-12-29, Close, 169.20
2021